In [538]:
from cayman_office_system import *

In [662]:
KEYS_TRANSACTION = ['Type',
 'ISIN Code / Abbr. Code',
 'Security Description',
 'Total No. of Shares',
 'Average Price',
 'Considerations',
 'Commission',
 'Sales Tax',
 'Capital Gains Tax',
 'Net Amount']

KEY_SELLBUY = 'No. of Shares / Price'

import re
from datetime import datetime

FILE_NAME_PREFIX_TRADE = 'LKEF Trade'

def get_date_from_file_name(file_name, form):
    match = re.search(r'\d{8}', file_name)
    if not match:
        return None
    
    date_str = match.group()
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    
    return date_obj.strftime(form)

def get_dates_of_trades_in_file_folder(file_folder=file_folder['trade'], form='%Y-%m-%d'):
    file_names = scan_files_including_regex(file_folder=file_folder, regex='LKEF Trade')
    dates = [get_date_from_file_name(file_name=file_name, form=form) for file_name in file_names]
    return dates

def open_df_trade_by_date(date, file_folder=file_folder['trade'], verbose=False):
    date = date.replace('-', '')
    regex = FILE_NAME_PREFIX_TRADE+f'.*{date}'
    file_name = scan_files_including_regex(file_folder=file_folder, regex=regex, date=date)[-1]
    df = open_excel(file_folder=file_folder, file_name=file_name, engine='xlrd')
    if verbose:
        print('File Name: ', file_name)
    return df

def open_df_trade_by_index(index, file_folder=file_folder['trade'], verbose=False):
    regex = FILE_NAME_PREFIX_TRADE
    file_name = scan_files_including_regex(file_folder=file_folder, regex=regex)[index]
    df = open_excel(file_folder=file_folder, file_name=file_name, engine='xlrd')
    if verbose:
        print('File Name: ', file_name)
    return df

def get_keys_from_df_trade(df):
    keys = sorted(list(df['Unnamed: 0'].dropna()))
    return keys

def get_data_indicies_of_keys(keys, df):
    dct_indices = {}
    for key in keys:
        indices_of_key = list(df[df['Unnamed: 0']==key].index)
        dct_indices[key] = indices_of_key
    return dct_indices
    
def get_pair_index_of_info(dct_indices):
    index_i = 0
    index_f = dct_indices['Type'][0]-1
    return (index_i, index_f)

def get_pairs_index_of_transaction(dct_indices):
    indices_i = dct_indices['Type']
    indices_f = dct_indices['Net Amount']
    pairs_index = [(index_i, index_f+1) for index_i, index_f in zip(indices_i, indices_f)]
    return pairs_index

def get_df_raw_info(df, pair_info):
    index_i, index_f = pair_info
    df_info = df.iloc[index_i:index_f]
    return df_info

def get_df_raw_transaction(df, pair_raw_transaction):
    index_i, index_f = pair_raw_transaction
    df_raw_transaction = df.iloc[index_i:index_f]
    return df_raw_transaction

def get_df_transaction_by_index(df, index, pairs_transaction):
    pair_transaction = pairs_transaction[index]
    df_transaction = get_df_raw_transaction(df, pair_transaction)
    return df_transaction

# def get_dfs_transaction(df, pairs_trade):
#     dfs = {}
#     for index in range(len(pairs_trade)):
#         df_trade = get_df_transaction_by_index(df, index, pairs_trade)
#         dfs[index] = df_trade
#     return dfs

def get_dfs_transaction(df, pairs_transaction):
    dfs = []
    for i, pair in enumerate(pairs_transaction):
        df_transaction = get_df_transaction_by_index(df, i, pair)
        dfs.append(df_transaction)
    return dfs

def get_row_in_transaction(transaction, key):
    row = transaction[transaction['Unnamed: 0']==key]
    return row

def get_values_of_key_in_transaction(transaction, key):
    row = get_row_in_transaction(transaction, key)
    srs = row.dropna(axis=1).iloc[0]
    values = list(srs[1:])
    return values

def get_data_in_transaction(transaction, keys):
    dct = {}
    for key in keys:
        values = get_values_of_key_in_transaction(transaction, key)
        dct[key] = values
    return dct

def get_df_sellbuy(transaction):
    df = transaction[~transaction['Unnamed: 6'].isna()].dropna(axis=1)
    df.columns = ['num_shares', 'currency', 'price_executed']
    return df

def get_ticker_in_transaction(data):
    isin_code, abbr_code = data['ISIN Code / Abbr. Code'][-1].split('/')
    isin_code, abbr_code = isin_code.strip(), abbr_code.strip()
    ticker = f'{isin_code[3:-3]} KS'
    return ticker

def get_type_in_transaction(data):
    return data['Type'][-1]

def get_isin_and_abbr_code_in_transaction(data):
    isin_code, abbr_code = data['ISIN Code / Abbr. Code'][-1].split('/')
    isin_code, abbr_code = isin_code.strip(), abbr_code.strip()
    return isin_code, abbr_code

def get_isin_code_in_transaction(data):
    return get_isin_and_abbr_code_in_transaction(data)[0]

def get_abbr_code_in_transaction(data):
    return get_isin_and_abbr_code_in_transaction(data)[1]

def get_ticker_in_transaction(data):
    isin_code = get_isin_code_in_transaction(data)
    ticker = f'{isin_code[3:-3]} KS'
    return ticker

def get_name_in_transaction(data):
    return data['Security Description'][-1]

def get_consideration_in_transaction(data):
    return data['Considerations'][-1]

def get_commission_in_transaction(data):
    return data['Commission'][-1]

def get_sales_tax_in_transaction(data):
    return data['Sales Tax'][-1]

def get_capital_gains_tax_in_transaction(data):
    return data['Capital Gains Tax'][-1]

def get_net_amount_in_transaction(data):
    return data['Net Amount'][-1]

def get_total_no_of_shares_in_transaction(data):
    return data['Total No. of Shares'][-1]

def get_average_price_in_transaction(data):
    return data['Average Price'][-1]
    

In [755]:
class Trade:
    def __init__(self, date=None, index=None, file_folder=file_folder['trade']):
        self.file_folder = file_folder
        self.date = date
        self.index = index
        self.set_date_and_index(date, index)
        self.raw = self.open_raw()
        self.keys = get_keys_from_df_trade(self.raw)
        self.indices = get_data_indicies_of_keys(self.keys, self.raw)
        self.pair_info = get_pair_index_of_info(self.indices)
        self.pairs_transaction = get_pairs_index_of_transaction(self.indices)
        self.transactions = self.get_transactions()


    def set_date_and_index(self, date, index):
        if date and index:
            raise ValueError('Both date and index are given.')
        elif date:
            self.date = date
            self.index = None
        elif index:
            self.index = index
            dates = get_dates_of_trades_in_file_folder(file_folder=self.file_folder, form='%Y-%m-%d')
            self.date = dates[self.index]
        else:
            self.index = -1
            dates = get_dates_of_trades_in_file_folder(file_folder=self.file_folder, form='%Y-%m-%d')
            self.date = dates[self.index]

        print(f'- set (date, index) = ({self.date}, {self.index})')
        return self

    def open_raw(self):
        if self.index:
            df = open_df_trade_by_index(self.index, file_folder=self.file_folder)
        elif self.date:
            df = open_df_trade_by_date(self.date, file_folder=self.file_folder)
        self.raw = df
        return df
    
    def get_raw_info(self):
        df_info = get_df_raw_info(self.raw, self.pair_info)
        return df_info
    
    def get_raws_tranaction(self):
        dfs = [get_df_raw_transaction(self.raw, pair) for pair in self.pairs_transaction]
        self.raws_transaction = dfs        
        return dfs
    
    def get_transactions(self):
        if not hasattr(self, 'raws_transaction'):
            self.get_raws_tranaction()
        transactions = [Transaction(raw, self.date) for raw in self.raws_transaction]
        self.transactions = transactions
        return transactions

    
class Transaction:
    def __init__(self, raw, date):
        self.raw = raw
        self.date = date
        self.info = self.get_data()
        self.get_properties()
        self.ticker = self.get_ticker()
        self.df_sellbuy = self.get_df_sellbuy()
        self.data = self.get_data_calculated()
        
    def get_data(self):
        data = get_data_in_transaction(self.raw, KEYS_TRANSACTION)
        self.info = data
        return data
    
    def get_properties(self):
        data = self.info
        self.type = get_type_in_transaction(data)
        self.name = get_name_in_transaction(data)
        self.isin_code = get_isin_code_in_transaction(data)
        self.abbr_code = get_abbr_code_in_transaction(data)
        self.consideration = get_consideration_in_transaction(data)
        self.commission = get_commission_in_transaction(data)
        self.sales_tax = get_sales_tax_in_transaction(data)
        self.capital_gain_tax = get_capital_gains_tax_in_transaction(data)
        self.net_amount = get_net_amount_in_transaction(data)
        self.num_shares = get_total_no_of_shares_in_transaction(data)
        self.average_price = get_average_price_in_transaction(data)
        return self
    
    def get_ticker(self):
        ticker = get_ticker_in_transaction(self.info)
        self.ticker = ticker
        return ticker
    
    def get_df_sellbuy(self):
        df = get_df_sellbuy(self.raw)
        df['date'] = self.date
        df['ticker'] = self.ticker
        df['amount'] = df['num_shares'] * df['price_executed']
        cols_ordered = ['date', 'ticker', 'num_shares', 'currency', 'price_executed', 'amount']
        df = df[cols_ordered]
        df = df.set_index('date')
        self.df_sellbuy = df
        return df
    
    def get_num_shares_calculated(self):
        num_shares = self.df_sellbuy['num_shares'].sum()
        self.num_shares_calculated = num_shares
        return num_shares

    def get_consideration_calculated(self):
        consideration = self.df_sellbuy['amount'].sum()
        self.consideration_calculated = consideration
        return consideration
    
    def get_commission_calculated(self, rnd=True):
        consideration = self.consideration_calculated
        commission = consideration/1000
        if rnd:
            commission = round(commission)
        self.commission_calculated = commission
        return commission
    
    def get_average_price_calculated(self):
        if not hasattr(self, 'consieration_calculated'):
            self.get_consideration_calculated()
        if not hasattr(self, 'num_shares_calculated'):
            self.get_num_shares_calculated()
        average_price = self.consideration_calculated/self.num_shares_calculated
        self.average_price_calculated = average_price
        return average_price
    
    def get_net_amount_calculated(self):
        if not hasattr(self, 'consieration_calculated'):
            self.get_consideration_calculated()
        if not hasattr(self, 'commission_calculated'):
            self.get_commission_calculated()
        net_amount = self.consideration_calculated + self.commission_calculated
        self.net_amount_calculated = net_amount
        return net_amount
    
    def get_data_calculated(self):
        if not hasattr(self, 'df_sellbuy'):
            self.get_df_sellbuy()
        self.get_num_shares_calculated()
        self.get_consideration_calculated()
        self.get_commission_calculated()
        self.get_average_price_calculated()
        self.get_net_amount_calculated()
        data = {
            'num_shares': self.num_shares_calculated,
            'consideration': self.consideration_calculated,
            'commission': self.commission_calculated,
            'average_price': self.average_price_calculated,
            'net_amount': self.net_amount_calculated
        }
        self.data_calculated = data
        return data

    def check_identity_of_data(self):
        print(f'Check Identity of Data: (system) == (calculated)')
        if self.num_shares == self.num_shares_calculated:
            print(f'- num_shares: {self.num_shares} == {self.num_shares_calculated}')
        else:
            print(f'- num_shares: {self.num_shares} != {self.num_shares_calculated}')
        if self.consideration == self.consideration_calculated:
            print(f'- consideration: {self.consideration} == {self.consideration_calculated}')
        else:    
            print(f'- consideration: {self.consideration} != {self.consideration_calculated}')
        if self.commission == self.commission_calculated:
            print(f'- commission: {self.commission} == {self.commission_calculated}')
        else:
            print(f'- commission: {self.commission} != {self.commission_calculated}')
        if self.average_price == self.average_price_calculated:
            print(f'- average_price: {self.average_price} == {self.average_price_calculated}')
        else:
            print(f'- average_price: {self.average_price} != {self.average_price_calculated}')
        if self.net_amount == self.net_amount_calculated:
            print(f'- net_amount: {self.net_amount} == {self.net_amount_calculated}')
        else:
            print(f'- net_amount: {self.net_amount} != {self.net_amount_calculated}')
        return None
    
    def show_info(self):
        self.check_identity_of_data()
        print(self.get_data_calculated())
        return self.df_sellbuy


In [756]:
tr = Trade()

- set (date, index) = (2024-09-09, -1)


In [760]:
tr.transactions[2].show_info()

Check Identity of Data: (system) == (calculated)
- num_shares: 1000 == 1000
- consideration: 88238200 == 88238200.0
- commission: 88238 == 88238
- average_price: 88238.2 == 88238.2
- net_amount: 88326438 == 88326438.0
{'num_shares': 1000, 'consideration': 88238200.0, 'commission': 88238, 'average_price': 88238.2, 'net_amount': 88326438.0}


,ticker,num_shares,currency,price_executed,amount
date,,,,,
2024-09-09,271560 KS,43,/ KRW,87400.0,3758200.0
2024-09-09,271560 KS,90,/ KRW,87500.0,7875000.0
2024-09-09,271560 KS,81,/ KRW,87600.0,7095600.0
2024-09-09,271560 KS,23,/ KRW,87700.0,2017100.0
2024-09-09,271560 KS,23,/ KRW,87800.0,2019400.0
2024-09-09,271560 KS,75,/ KRW,87900.0,6592500.0
2024-09-09,271560 KS,25,/ KRW,88000.0,2200000.0
2024-09-09,271560 KS,71,/ KRW,88100.0,6255100.0
2024-09-09,271560 KS,37,/ KRW,88200.0,3263400.0


In [739]:
trxs = tr.get_raws_tranaction()
trxs

[                Unnamed: 0 Unnamed: 1 Unnamed: 2              Unnamed: 3  \
 20                    Type        NaN        NaN                     KSE   
 21  ISIN Code / Abbr. Code        NaN        NaN  KR7003030004 / A003030   
 22    Security Description        NaN        NaN     SeAH Steel Holdings   
 23                     NaN        NaN        NaN                     NaN   
 24   No. of Shares / Price        NaN        NaN                     NaN   
 25                     NaN        NaN        NaN                     NaN   
 26                     NaN        NaN        NaN                     NaN   
 27                     NaN        NaN        NaN                     NaN   
 28                     NaN        NaN        NaN                     NaN   
 29                     NaN        NaN        NaN                     NaN   
 30                     NaN        NaN        NaN                     NaN   
 31                     NaN        NaN        NaN                     NaN   

In [740]:
trxs[0]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,NaN,SeAH Steel Holdings,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0


In [741]:
trxs[0]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,NaN,SeAH Steel Holdings,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0


In [742]:
tr.date

'2024-09-09'

In [743]:
trx = Transaction(raw=trxs[0], date=tr.date)
trx

In [744]:
trx.ticker

'003030 KS'

In [745]:
trx.df_sellbuy

,ticker,num_shares,currency,price_executed,amount
date,,,,,
2024-09-09,003030 KS,1,/ KRW,172600.0,172600.0
2024-09-09,003030 KS,15,/ KRW,173200.0,2598000.0
2024-09-09,003030 KS,1,/ KRW,173300.0,173300.0
2024-09-09,003030 KS,5,/ KRW,173400.0,867000.0
2024-09-09,003030 KS,6,/ KRW,173500.0,1041000.0
2024-09-09,003030 KS,7,/ KRW,173600.0,1215200.0
2024-09-09,003030 KS,37,/ KRW,173700.0,6426900.0
2024-09-09,003030 KS,37,/ KRW,173800.0,6430600.0
2024-09-09,003030 KS,4,/ KRW,173900.0,695600.0


In [746]:
trx.info

{'Type': ['KSE', 'Buy'],
 'ISIN Code / Abbr. Code': ['KR7003030004 / A003030'],
 'Security Description': ['SeAH Steel Holdings'],
 'Total No. of Shares': ['Shs', 500],
 'Average Price': ['KRW', 174271],
 'Considerations': ['KRW', 87135500],
 'Commission': ['KRW', 87136],
 'Sales Tax': ['KRW', 0],
 'Capital Gains Tax': ['KRW', 0],
 'Net Amount': ['KRW', 87222636]}

In [747]:
trx.get_data_calculated()

{'num_shares': 500,
 'consideration': 87135500.0,
 'commission': 87136,
 'average_price': 174271.0,
 'net_amount': 87222636.0}

In [748]:
trx.check_identity_of_data()

Check Identity of Data: (system) == (calculated)
- num_shares: 500 == 500
- consideration: 87135500 == 87135500.0
- commission: 87136 == 87136
- average_price: 174271 == 174271.0
- net_amount: 87222636 == 87222636.0


In [733]:
trx = Transaction(raw=trxs[1], date=tr.date)
trx.df_sellbuy


,ticker,num_shares,currency,price_executed,amount
date,,,,,
2024-09-09,007340 KS,27,/ KRW,91900.0,2481300.0
2024-09-09,007340 KS,37,/ KRW,92000.0,3404000.0
2024-09-09,007340 KS,36,/ KRW,92100.0,3315600.0
2024-09-09,007340 KS,17,/ KRW,92200.0,1567400.0
2024-09-09,007340 KS,155,/ KRW,92300.0,14306500.0
2024-09-09,007340 KS,6,/ KRW,92400.0,554400.0
2024-09-09,007340 KS,70,/ KRW,92500.0,6475000.0
2024-09-09,007340 KS,1,/ KRW,92700.0,92700.0
2024-09-09,007340 KS,6,/ KRW,92800.0,556800.0


In [734]:
trx.check_identity_of_data()

- num_shares: 2231 == 2231
- consideration: 208724800 == 208724800.0
- commission: 208725 == 208725
- average_price: 93556.6114 != 93556.61138502913
- net_amount: 208933525 == 208933525.0


In [735]:
trx = Transaction(raw=trxs[2], date=tr.date)
trx.df_sellbuy

,ticker,num_shares,currency,price_executed,amount
date,,,,,
2024-09-09,271560 KS,43,/ KRW,87400.0,3758200.0
2024-09-09,271560 KS,90,/ KRW,87500.0,7875000.0
2024-09-09,271560 KS,81,/ KRW,87600.0,7095600.0
2024-09-09,271560 KS,23,/ KRW,87700.0,2017100.0
2024-09-09,271560 KS,23,/ KRW,87800.0,2019400.0
2024-09-09,271560 KS,75,/ KRW,87900.0,6592500.0
2024-09-09,271560 KS,25,/ KRW,88000.0,2200000.0
2024-09-09,271560 KS,71,/ KRW,88100.0,6255100.0
2024-09-09,271560 KS,37,/ KRW,88200.0,3263400.0


In [736]:
trx.check_identity_of_data()

- num_shares: 1000 == 1000
- consideration: 88238200 == 88238200.0
- commission: 88238 == 88238
- average_price: 88238.2 == 88238.2
- net_amount: 88326438 == 88326438.0


In [660]:
data = trx.info
data

{'Type': ['KSE', 'Buy'],
 'ISIN Code / Abbr. Code': ['KR7003030004 / A003030'],
 'Security Description': ['SeAH Steel Holdings'],
 'Total No. of Shares': ['Shs', 500],
 'Average Price': ['KRW', 174271],
 'Considerations': ['KRW', 87135500],
 'Commission': ['KRW', 87136],
 'Sales Tax': ['KRW', 0],
 'Capital Gains Tax': ['KRW', 0],
 'Net Amount': ['KRW', 87222636]}

In [639]:
trx.ticker

'003030 KS'

In [640]:
df = trx.get_df_sellbuy()
df

,ticker,num_shares,currency,price_executed,amount
date,,,,,
2024-09-09,003030 KS,1,/ KRW,172600.0,172600.0
2024-09-09,003030 KS,15,/ KRW,173200.0,2598000.0
2024-09-09,003030 KS,1,/ KRW,173300.0,173300.0
2024-09-09,003030 KS,5,/ KRW,173400.0,867000.0
2024-09-09,003030 KS,6,/ KRW,173500.0,1041000.0
2024-09-09,003030 KS,7,/ KRW,173600.0,1215200.0
2024-09-09,003030 KS,37,/ KRW,173700.0,6426900.0
2024-09-09,003030 KS,37,/ KRW,173800.0,6430600.0
2024-09-09,003030 KS,4,/ KRW,173900.0,695600.0


In [645]:
df['amount'].sum()

87135500.0

In [659]:
round(87135500.0/1000)

87136

In [654]:
87135500.0/1000 * 100


8713550.0

In [527]:
tr.raw

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN,NaN,NaN,NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN,NaN,NaN,NaN,NaN
3,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Please notify us by email (middle.st@samsung.c...,NaN,NaN,NaN,NaN,NaN,NaN
131,in this confirmation by 5pm (Korea time) on th...,NaN,NaN,NaN,NaN,NaN,NaN
132,Note we shall not be liable for any indirect o...,NaN,NaN,NaN,NaN,NaN,NaN


In [528]:
tr.pair_info

(0, 19)

In [529]:
tr.get_raw_info()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN,NaN,NaN,NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN,NaN,NaN,NaN,NaN
3,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Trade Confirmation,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Date,09 Sep. 2024,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,From,"Samsung Securities Co., Ltd.",NaN,NaN,NaN,NaN,NaN


In [530]:
tr.pairs_transaction

[(20, 52), (53, 86), (87, 117)]

In [532]:
dfs = tr.get_raws_tranaction()

In [535]:
dfs[2]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
87,Type,NaN,NaN,KSE,Buy,NaN,NaN
88,ISIN Code / Abbr. Code,NaN,NaN,KR7271560005 / A271560,NaN,NaN,NaN
89,Security Description,NaN,NaN,ORION,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,No. of Shares / Price,NaN,NaN,NaN,43,/ KRW,87400.0
92,NaN,NaN,NaN,NaN,90,/ KRW,87500.0
93,NaN,NaN,NaN,NaN,81,/ KRW,87600.0
94,NaN,NaN,NaN,NaN,23,/ KRW,87700.0
95,NaN,NaN,NaN,NaN,23,/ KRW,87800.0
96,NaN,NaN,NaN,NaN,75,/ KRW,87900.0


In [524]:
get_df_transaction_by_index(tr.raw, 0, tr.pairs_transaction)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,NaN,SeAH Steel Holdings,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0


In [479]:
tr.num_transaction

3

In [480]:
tr.ids_transaction

[0, 1, 2]

In [481]:
transaction = dct[0]
transaction

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,NaN,SeAH Steel Holdings,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0


In [496]:
def get_df_in_transaction(transaction, key):
    df = transaction[transaction['Unnamed: 0']==key]
    return df

def get_values_of_key_in_transaction(transaction, key):
    df = get_df_in_transaction(transaction, key)
    srs = df.dropna(axis=1).iloc[0]
    values = list(srs[1:])
    return values

def get_data_in_transaction(transaction, keys):
    dct = {}
    for key in keys:
        values = get_values_of_key_in_transaction(transaction, key)
        dct[key] = values
    return dct

def get_df_sellbuy(transaction):
    df = transaction[~transaction['Unnamed: 6'].isna()].dropna(axis=1)
    df.columns = ['amount', 'currency', 'price_executed']
    return df

def get_ticker_in_transaction(data):
    isin_code, abbr_code = data['ISIN Code / Abbr. Code'][-1].split('/')
    isin_code, abbr_code = isin_code.strip(), abbr_code.strip()
    ticker = f'{isin_code[3:-3]} KS'
    return ticker




In [497]:
get_df_sellbuy(transaction)

,amount,currency,price_executed
24,1,/ KRW,172600.0
25,15,/ KRW,173200.0
26,1,/ KRW,173300.0
27,5,/ KRW,173400.0
28,6,/ KRW,173500.0
29,7,/ KRW,173600.0
30,37,/ KRW,173700.0
31,37,/ KRW,173800.0
32,4,/ KRW,173900.0
33,24,/ KRW,174000.0


In [498]:
data = get_data_in_transaction(transaction, KEYS_TRANSACTION)
data

{'Type': ['KSE', 'Buy'],
 'ISIN Code / Abbr. Code': ['KR7003030004 / A003030'],
 'Security Description': ['SeAH Steel Holdings'],
 'Total No. of Shares': ['Shs', 500],
 'Average Price': ['KRW', 174271],
 'Considerations': ['KRW', 87135500],
 'Commission': ['KRW', 87136],
 'Sales Tax': ['KRW', 0],
 'Capital Gains Tax': ['KRW', 0],
 'Net Amount': ['KRW', 87222636]}

In [499]:
ticker = get_ticker_in_transaction(data)
ticker

'003030 KS'

In [503]:
df = get_df_sellbuy(transaction)
data = get_data_in_transaction(transaction, KEYS_TRANSACTION)
df['ticker'] = get_ticker_in_transaction(data)
df = df.set_index('ticker')
df

,amount,currency,price_executed
ticker,,,
003030 KS,1,/ KRW,172600.0
003030 KS,15,/ KRW,173200.0
003030 KS,1,/ KRW,173300.0
003030 KS,5,/ KRW,173400.0
003030 KS,6,/ KRW,173500.0
003030 KS,7,/ KRW,173600.0
003030 KS,37,/ KRW,173700.0
003030 KS,37,/ KRW,173800.0
003030 KS,4,/ KRW,173900.0


In [485]:
transaction[~transaction['Unnamed: 6'].isna()].dropna(axis=1)

,Unnamed: 4,Unnamed: 5,Unnamed: 6
24,1,/ KRW,172600.0
25,15,/ KRW,173200.0
26,1,/ KRW,173300.0
27,5,/ KRW,173400.0
28,6,/ KRW,173500.0
29,7,/ KRW,173600.0
30,37,/ KRW,173700.0
31,37,/ KRW,173800.0
32,4,/ KRW,173900.0
33,24,/ KRW,174000.0


In [462]:
get_data_in_transaction(transaction,  'Security Description')

['SeAH Steel Holdings']

In [425]:
list(transaction['Unnamed: 0'].dropna())

['Type',
 'ISIN Code / Abbr. Code',
 'Security Description',
 'No. of Shares / Price',
 'Total No. of Shares',
 'Average Price',
 'Considerations',
 'Commission',
 'Sales Tax',
 'Capital Gains Tax',
 'Net Amount']

In [409]:
keys_in_transaction = []
for key, indices in tr.indices.items():
    if len(indices) == tr.num_transaction:
        keys_in_transaction.append(key)
keys_in_transaction


['Average Price',
 'Capital Gains Tax',
 'Commission',
 'Considerations',
 'ISIN Code / Abbr. Code',
 'Net Amount',
 'No. of Shares / Price',
 'Sales Tax',
 'Security Description',
 'Total No. of Shares',
 'Type']

In [410]:
keys_in_transaction

['Average Price',
 'Capital Gains Tax',
 'Commission',
 'Considerations',
 'ISIN Code / Abbr. Code',
 'Net Amount',
 'No. of Shares / Price',
 'Sales Tax',
 'Security Description',
 'Total No. of Shares',
 'Type']

In [421]:
transaction[transaction['Unnamed: 0']=='Net Amount'].dropna(axis=1)

,Unnamed: 0,Unnamed: 2,Unnamed: 4
51,Net Amount,KRW,87222636


In [371]:
transaction[transaction['Unnamed: 0']=='Sales Tax'].dropna(axis=1)

,Unnamed: 0,Unnamed: 2,Unnamed: 4
49,Sales Tax,KRW,0


In [356]:
tr.indices['ISIN Code / Abbr. Code']

[21, 54, 88]

In [340]:
transaction[0]

KeyError: 0

In [331]:
tr.dct_indices

{' Trade Confirmation': [5],
 '*Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.': [126],
 '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ': [2],
 '9th Fl. Samsung Electronics Bldg., ': [1],
 'A/C Name :': [18],
 'Average Price': [46, 80, 111],
 'BIC Code : SAMCKRS1': [125],
 'Capital Gains Tax': [50, 84, 115],
 'Commission': [48, 82, 113],
 'Considerations': [47, 81, 112],
 'Date': [7],
 'From': [9],
 'ISIN Code / Abbr. Code': [21, 54, 88],
 'KSD Participant Number : 30-0000': [124],
 'Korea IRC No. :': [17],
 'Net Amount': [51, 85, 116],
 'No. of Shares / Price': [24, 57, 91],
 'Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges ': [132],
 'Our Reference No. :': [16],
 'PSET BIC : KSDCKRSE': [123],
 'Place of Settlement (PSET) : KSD (Korea Securities Depository)': [122],
 'Please notify us by email (middle.st@samsung.com/ settle.clearing@samsung.com) of any po

In [309]:
isin_abbr_code = tr.dct_indices['ISIN Code / Abbr. Code']
isin_abbr_code

[21, 54, 88]

In [317]:
isin_abbr_code = tr.dct_indices['ISIN Code / Abbr. Code']
lst = list(tr.raw.loc[isin_abbr_code[0]].dropna())
isin_code, abbr_code = lst[-1].split('/')
isin_code, abbr_code = isin_code.strip(), abbr_code.strip()
isin_code, abbr_code

('KR7003030004', 'A003030')

In [302]:
dct[1]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
53,Type,NaN,NaN,KSE,Buy,NaN,NaN
54,ISIN Code / Abbr. Code,NaN,NaN,KR7007340003 / A007340,NaN,NaN,NaN
55,Security Description,NaN,NaN,DN AUTOMOTIVE,NaN,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,No. of Shares / Price,NaN,NaN,NaN,27,/ KRW,91900.0
58,NaN,NaN,NaN,NaN,37,/ KRW,92000.0
59,NaN,NaN,NaN,NaN,36,/ KRW,92100.0
60,NaN,NaN,NaN,NaN,17,/ KRW,92200.0
61,NaN,NaN,NaN,NaN,155,/ KRW,92300.0
62,NaN,NaN,NaN,NaN,6,/ KRW,92400.0


In [238]:
regex = FILE_NAME_PREFIX_TRADE
file_name = scan_files_including_regex(file_folder=file_folder['trade'], regex=regex)[-1]
file_name



'LKEF Trade 20240909.xls'

In [239]:
file_name = scan_files_including_regex(file_folder=file_folder['trade'], regex=FILE_NAME_PREFIX_TRADE)[-1]
file_name

'LKEF Trade 20240909.xls'

In [240]:
df = open_excel(file_folder=file_folder['trade'], file_name=file_name, engine='xlrd')
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN,NaN,NaN,NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN,NaN,NaN,NaN,NaN
3,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Please notify us by email (middle.st@samsung.c...,NaN,NaN,NaN,NaN,NaN,NaN
131,in this confirmation by 5pm (Korea time) on th...,NaN,NaN,NaN,NaN,NaN,NaN
132,Note we shall not be liable for any indirect o...,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
get_dates_of_trades_in_file_folder()

['2024-09-02', '2024-09-05', '2024-09-06', '2024-09-09']

In [244]:
df = open_df_trade_by_index(index=-2)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN,NaN,NaN,NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN,NaN,NaN,NaN,NaN
3,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Trade Confirmation,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Date,06 Sep. 2024,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,From,"Samsung Securities Co., Ltd.",NaN,NaN,NaN,NaN,NaN


In [243]:
file_names = scan_files_including_regex(file_folder=file_folder['trade'], regex='LKEF Trade')
file_names

['LKEF Trade 20240902.xls',
 'LKEF Trade 20240905.xls',
 'LKEF Trade 20240906.xls',
 'LKEF Trade 20240909.xls']

In [ ]:
df = open_excel(file_folder=file_folder['trade'], file_name=file_name, engine='xlrd')
df

In [187]:
df = open_df_trade_by_index(index=-1, verbose=True)
df

File Name:  LKEF Trade 20240909.xls


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN,NaN,NaN,NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN,NaN,NaN,NaN,NaN
3,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Please notify us by email (middle.st@samsung.c...,NaN,NaN,NaN,NaN,NaN,NaN
131,in this confirmation by 5pm (Korea time) on th...,NaN,NaN,NaN,NaN,NaN,NaN
132,Note we shall not be liable for any indirect o...,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
keys = get_keys_from_df_trade(df)
keys

[' Trade Confirmation',
 '*Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.',
 '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ',
 '9th Fl. Samsung Electronics Bldg., ',
 'A/C Name :',
 'Average Price',
 'Average Price',
 'Average Price',
 'BIC Code : SAMCKRS1',
 'Capital Gains Tax',
 'Capital Gains Tax',
 'Capital Gains Tax',
 'Commission',
 'Commission',
 'Commission',
 'Considerations',
 'Considerations',
 'Considerations',
 'Date',
 'From',
 'ISIN Code / Abbr. Code',
 'ISIN Code / Abbr. Code',
 'ISIN Code / Abbr. Code',
 'KSD Participant Number : 30-0000',
 'Korea IRC No. :',
 'Net Amount',
 'Net Amount',
 'Net Amount',
 'No. of Shares / Price',
 'No. of Shares / Price',
 'No. of Shares / Price',
 'Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges ',
 'Our Reference No. :',
 'PSET BIC : KSDCKRSE',
 'Place of Settlement (PSET) : KSD (Korea Securities Deposito

In [189]:
dct_indices = get_data_indicies_of_keys(keys, df)
dct_indices

{' Trade Confirmation': [5],
 '*Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.': [126],
 '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ': [2],
 '9th Fl. Samsung Electronics Bldg., ': [1],
 'A/C Name :': [18],
 'Average Price': [46, 80, 111],
 'BIC Code : SAMCKRS1': [125],
 'Capital Gains Tax': [50, 84, 115],
 'Commission': [48, 82, 113],
 'Considerations': [47, 81, 112],
 'Date': [7],
 'From': [9],
 'ISIN Code / Abbr. Code': [21, 54, 88],
 'KSD Participant Number : 30-0000': [124],
 'Korea IRC No. :': [17],
 'Net Amount': [51, 85, 116],
 'No. of Shares / Price': [24, 57, 91],
 'Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges ': [132],
 'Our Reference No. :': [16],
 'PSET BIC : KSDCKRSE': [123],
 'Place of Settlement (PSET) : KSD (Korea Securities Depository)': [122],
 'Please notify us by email (middle.st@samsung.com/ settle.clearing@samsung.com) of any po

In [190]:
pairs_trade = get_pairs_index_of_trades(dct_indices)
pair_info = get_pair_index_of_info(dct_indices)

In [191]:
pairs_trade, pair_info

([(20, 51), (53, 85), (87, 116)], (0, 19))

In [192]:
df.loc[pairs_trade[0][0]:pairs_trade[0][1], :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,NaN,SeAH Steel Holdings,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0


In [193]:
df.loc[pairs_trade[1][0]:pairs_trade[1][1], :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
53,Type,NaN,NaN,KSE,Buy,NaN,NaN
54,ISIN Code / Abbr. Code,NaN,NaN,KR7007340003 / A007340,NaN,NaN,NaN
55,Security Description,NaN,NaN,DN AUTOMOTIVE,NaN,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,No. of Shares / Price,NaN,NaN,NaN,27,/ KRW,91900.0
58,NaN,NaN,NaN,NaN,37,/ KRW,92000.0
59,NaN,NaN,NaN,NaN,36,/ KRW,92100.0
60,NaN,NaN,NaN,NaN,17,/ KRW,92200.0
61,NaN,NaN,NaN,NaN,155,/ KRW,92300.0
62,NaN,NaN,NaN,NaN,6,/ KRW,92400.0


In [195]:
df.loc[pairs_trade[2][0]:pairs_trade[2][1], :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
87,Type,NaN,NaN,KSE,Buy,NaN,NaN
88,ISIN Code / Abbr. Code,NaN,NaN,KR7271560005 / A271560,NaN,NaN,NaN
89,Security Description,NaN,NaN,ORION,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,No. of Shares / Price,NaN,NaN,NaN,43,/ KRW,87400.0
92,NaN,NaN,NaN,NaN,90,/ KRW,87500.0
93,NaN,NaN,NaN,NaN,81,/ KRW,87600.0
94,NaN,NaN,NaN,NaN,23,/ KRW,87700.0
95,NaN,NaN,NaN,NaN,23,/ KRW,87800.0
96,NaN,NaN,NaN,NaN,75,/ KRW,87900.0


In [196]:
df.loc[pair_info[0]:pair_info[1], :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN,NaN,NaN,NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN,NaN,NaN,NaN,NaN
3,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Trade Confirmation,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Date,09 Sep. 2024,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,From,"Samsung Securities Co., Ltd.",NaN,NaN,NaN,NaN,NaN


In [175]:
indices_i = dct_keys['Type']
indices_f = dct_keys['Net Amount']

In [176]:
df.loc[indices_i[0]:indices_f[0]]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,NaN,SeAH Steel Holdings,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0


In [164]:
file_folder['trade']

'/Users/juneyoungpark/dev/module-cayman_office_system/cayman_office_system/dataset-trade'

In [4]:
file_names = scan_files_including_regex(file_folder=file_folder['trade'], regex='LKEF Trade')
file_names

['LKEF Trade 20240902.xls',
 'LKEF Trade 20240905.xls',
 'LKEF Trade 20240906.xls',
 'LKEF Trade 20240909.xls']

In [101]:
df = open_excel(file_folder=file_folder['trade'], file_name=file_names[-1], engine='xlrd')
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN,NaN,NaN,NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN,NaN,NaN,NaN,NaN
3,Republic of Korea,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Please notify us by email (middle.st@samsung.c...,NaN,NaN,NaN,NaN,NaN,NaN
131,in this confirmation by 5pm (Korea time) on th...,NaN,NaN,NaN,NaN,NaN,NaN
132,Note we shall not be liable for any indirect o...,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
keys = sorted(list(df['Unnamed: 0'].dropna()))
keys

[' Trade Confirmation',
 '*Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.',
 '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ',
 '9th Fl. Samsung Electronics Bldg., ',
 'A/C Name :',
 'Average Price',
 'Average Price',
 'Average Price',
 'BIC Code : SAMCKRS1',
 'Capital Gains Tax',
 'Capital Gains Tax',
 'Capital Gains Tax',
 'Commission',
 'Commission',
 'Commission',
 'Considerations',
 'Considerations',
 'Considerations',
 'Date',
 'From',
 'ISIN Code / Abbr. Code',
 'ISIN Code / Abbr. Code',
 'ISIN Code / Abbr. Code',
 'KSD Participant Number : 30-0000',
 'Korea IRC No. :',
 'Net Amount',
 'Net Amount',
 'Net Amount',
 'No. of Shares / Price',
 'No. of Shares / Price',
 'No. of Shares / Price',
 'Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges ',
 'Our Reference No. :',
 'PSET BIC : KSDCKRSE',
 'Place of Settlement (PSET) : KSD (Korea Securities Deposito

In [103]:
dct_indices = {}
for key in keys:
    indices_of_key = list(df[df['Unnamed: 0']==key].index)
    dct_indices[key] = indices_of_key
dct_indices

{' Trade Confirmation': [5],
 '*Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.': [126],
 '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ': [2],
 '9th Fl. Samsung Electronics Bldg., ': [1],
 'A/C Name :': [18],
 'Average Price': [46, 80, 111],
 'BIC Code : SAMCKRS1': [125],
 'Capital Gains Tax': [50, 84, 115],
 'Commission': [48, 82, 113],
 'Considerations': [47, 81, 112],
 'Date': [7],
 'From': [9],
 'ISIN Code / Abbr. Code': [21, 54, 88],
 'KSD Participant Number : 30-0000': [124],
 'Korea IRC No. :': [17],
 'Net Amount': [51, 85, 116],
 'No. of Shares / Price': [24, 57, 91],
 'Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges ': [132],
 'Our Reference No. :': [16],
 'PSET BIC : KSDCKRSE': [123],
 'Place of Settlement (PSET) : KSD (Korea Securities Depository)': [122],
 'Please notify us by email (middle.st@samsung.com/ settle.clearing@samsung.com) of any po

In [105]:
dct_n = {}
for k, indices in dct_indices.items():
    print('key:', k)
    n = len(indices)
    dct_n[k] = n
dct_n


key:  Trade Confirmation
key: *Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.
key: 11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, 
key: 9th Fl. Samsung Electronics Bldg., 
key: A/C Name :
key: Average Price
key: BIC Code : SAMCKRS1
key: Capital Gains Tax
key: Commission
key: Considerations
key: Date
key: From
key: ISIN Code / Abbr. Code
key: KSD Participant Number : 30-0000
key: Korea IRC No. :
key: Net Amount
key: No. of Shares / Price
key: Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges 
key: Our Reference No. :
key: PSET BIC : KSDCKRSE
key: Place of Settlement (PSET) : KSD (Korea Securities Depository)
key: Please notify us by email (middle.st@samsung.com/ settle.clearing@samsung.com) of any potential discrepancies or errors
key: Republic of Korea
key: Sales Tax
key: Samsung Securities's Settlement Instructions
key: Security Description
key: Settlement Da

{' Trade Confirmation': 1,
 '*Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.': 1,
 '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ': 1,
 '9th Fl. Samsung Electronics Bldg., ': 1,
 'A/C Name :': 1,
 'Average Price': 3,
 'BIC Code : SAMCKRS1': 1,
 'Capital Gains Tax': 3,
 'Commission': 3,
 'Considerations': 3,
 'Date': 1,
 'From': 1,
 'ISIN Code / Abbr. Code': 3,
 'KSD Participant Number : 30-0000': 1,
 'Korea IRC No. :': 1,
 'Net Amount': 3,
 'No. of Shares / Price': 3,
 'Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges ': 1,
 'Our Reference No. :': 1,
 'PSET BIC : KSDCKRSE': 1,
 'Place of Settlement (PSET) : KSD (Korea Securities Depository)': 1,
 'Please notify us by email (middle.st@samsung.com/ settle.clearing@samsung.com) of any potential discrepancies or errors': 1,
 'Republic of Korea': 1,
 'Sales Tax': 3,
 "Samsung Securities's Settlement Instructions

In [106]:
ns = sorted(set(sorted(dct_n.values())))
ns

[1, 3]

In [83]:
dct_k = {}
for n in ns:
    dct_k[n] = [k for k, v in dct_n.items() if v==n]
dct_k

{1: [' Trade Confirmation',
  '*Samsung Securities is a self-clearing house, the same BIC code for both executing and clearing.',
  '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ',
  '9th Fl. Samsung Electronics Bldg., ',
  'A/C Name :',
  'BIC Code : SAMCKRS1',
  'Date',
  'From',
  'KSD Participant Number : 30-0000',
  'Korea IRC No. :',
  'Note we shall not be liable for any indirect or consequential loss or for any buy-in or replacement costs, penalties or charges ',
  'Our Reference No. :',
  'PSET BIC : KSDCKRSE',
  'Place of Settlement (PSET) : KSD (Korea Securities Depository)',
  'Please notify us by email (middle.st@samsung.com/ settle.clearing@samsung.com) of any potential discrepancies or errors',
  'Republic of Korea',
  "Samsung Securities's Settlement Instructions",
  'Settlement Date :',
  'Settlement Method : Delivery Versus Payment',
  'To',
  'Trade Date :',
  'in this confirmation by 5pm (Korea time) on the business day following the trade date.',
  'incurred in 

In [152]:
indices_type = dct_indices['Type'] + [-1]
indices_type

[20, 53, 87, -1]

In [158]:
index_pairs_type = [(indices_type[i], indices_type[i+1]-1) for i in range(len(indices_type)-1)]
index_pairs_type

[(20, 52), (53, 86), (87, -2)]

In [163]:
df.loc[index_pairs_type[0][0]:index_pairs_type[0][1], :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,NaN,SeAH Steel Holdings,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0


In [93]:
keys_multi = list(dct_k.values())[-1]
keys_multi

['Average Price',
 'Capital Gains Tax',
 'Commission',
 'Considerations',
 'ISIN Code / Abbr. Code',
 'Net Amount',
 'No. of Shares / Price',
 'Sales Tax',
 'Security Description',
 'Total No. of Shares',
 'Type']

In [115]:
df[df['Unnamed: 0']==keys_multi[6]]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
49,Sales Tax,NaN,KRW,NaN,0,NaN,NaN
83,Sales Tax,NaN,KRW,NaN,0,NaN,NaN
114,Sales Tax,NaN,KRW,NaN,0,NaN,NaN


In [133]:
indices_of_num_of_shares = dct_indices[keys_multi[6]]
indices_of_num_of_shares

[24, 57, 91]

In [135]:
df[~df['Unnamed: 6'].isna()].loc[indices_of_num_of_shares[0]:indices_of_num_of_shares[1]-1, :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
24,No. of Shares / Price,NaN,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,NaN,7,/ KRW,173600.0
30,NaN,NaN,NaN,NaN,37,/ KRW,173700.0
31,NaN,NaN,NaN,NaN,37,/ KRW,173800.0
32,NaN,NaN,NaN,NaN,4,/ KRW,173900.0
33,NaN,NaN,NaN,NaN,24,/ KRW,174000.0


In [136]:
df[~df['Unnamed: 6'].isna()].loc[indices_of_num_of_shares[1]:indices_of_num_of_shares[2]-1, :]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
57,No. of Shares / Price,NaN,NaN,NaN,27,/ KRW,91900.0
58,NaN,NaN,NaN,NaN,37,/ KRW,92000.0
59,NaN,NaN,NaN,NaN,36,/ KRW,92100.0
60,NaN,NaN,NaN,NaN,17,/ KRW,92200.0
61,NaN,NaN,NaN,NaN,155,/ KRW,92300.0
62,NaN,NaN,NaN,NaN,6,/ KRW,92400.0
63,NaN,NaN,NaN,NaN,70,/ KRW,92500.0
64,NaN,NaN,NaN,NaN,1,/ KRW,92700.0
65,NaN,NaN,NaN,NaN,6,/ KRW,92800.0
66,NaN,NaN,NaN,NaN,24,/ KRW,92900.0


In [99]:
indices = dct[keys_multi[1]]
indices

[50, 84, 115]

In [100]:
df.iloc[indices[0]]

incurred in respect of this transaction.
Unnamed: 0    Capital Gains Tax
Unnamed: 1                  NaN
Unnamed: 2                  KRW
Unnamed: 3                  NaN
Unnamed: 4                    0
Unnamed: 5                  NaN
Unnamed: 6                  NaN
Name: 50, dtype: object

In [ ]:
for k, n in dct_n.items():
    for n in ns:
        if n == dct_n[k]:
            print(k, n)

In [73]:
bucket[6]

incurred in respect of this transaction.,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
125,BIC Code : SAMCKRS1,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
indices_division = [0] + list(df[df['Unnamed: 0']=='Type'].index) + [-1]
indices_division

[0, 20, 53, 87, -1]

In [29]:
def get_indices_division_of_sibject(df, subject):
    indices_division = [0] + list(df[df['Unnamed: 0']==subject].index) + [-1]
    return indices_division

In [24]:
df1 = df.iloc[indices_division[0]:indices_division[1], :]
df1 = df1.dropna(axis=0, how='all')
df1 = df1.dropna(axis=1, how='all')
df1

,Unnamed: 0,Unnamed: 1,Unnamed: 3
1,"9th Fl. Samsung Electronics Bldg.,",NaN,NaN
2,"11, 74-gil Seocho-daero Seocho-gu, Seoul 06620,",NaN,NaN
3,Republic of Korea,NaN,NaN
5,Trade Confirmation,NaN,NaN
7,Date,09 Sep. 2024,NaN
9,From,"Samsung Securities Co., Ltd.",NaN
10,To,OP INVESTMENT MANAGEMENT LIMITED,NaN
12,Trade Date :,NaN,09 Sep. 2024
13,Settlement Date :,NaN,11 Sep. 2024
16,Our Reference No. :,NaN,715543911501


In [25]:
df2 = df.iloc[indices_division[1]:indices_division[2], :]
df2 = df2.dropna(axis=0, how='all')
df2 = df2.dropna(axis=1, how='all')
df2

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
20,Type,NaN,KSE,Buy,NaN,NaN
21,ISIN Code / Abbr. Code,NaN,KR7003030004 / A003030,NaN,NaN,NaN
22,Security Description,NaN,SeAH Steel Holdings,NaN,NaN,NaN
24,No. of Shares / Price,NaN,NaN,1,/ KRW,172600.0
25,NaN,NaN,NaN,15,/ KRW,173200.0
26,NaN,NaN,NaN,1,/ KRW,173300.0
27,NaN,NaN,NaN,5,/ KRW,173400.0
28,NaN,NaN,NaN,6,/ KRW,173500.0
29,NaN,NaN,NaN,7,/ KRW,173600.0
30,NaN,NaN,NaN,37,/ KRW,173700.0


In [33]:
df = df2.T 
df.columns = df.iloc[0, :]
df = df.iloc[1:, :]
df

Unnamed: 0,Type,ISIN Code / Abbr. Code,Security Description,No. of Shares / Price,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Total No. of Shares,Average Price,Considerations,Commission,Sales Tax,Capital Gains Tax,Net Amount
Unnamed: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Shs,KRW,KRW,KRW,KRW,KRW,KRW
Unnamed: 3,KSE,KR7003030004 / A003030,SeAH Steel Holdings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,Buy,NaN,NaN,1,15,1,5,6,7,37,...,32,39,17,500,174271,87135500,87136,0,0,87222636
Unnamed: 5,NaN,NaN,NaN,/ KRW,/ KRW,/ KRW,/ KRW,/ KRW,/ KRW,/ KRW,...,/ KRW,/ KRW,/ KRW,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,NaN,NaN,NaN,172600.0,173200.0,173300.0,173400.0,173500.0,173600.0,173700.0,...,174800.0,174900.0,175000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df['Type'].dropna()

Unnamed: 3    KSE
Unnamed: 4    Buy
Name: Type, dtype: object

In [26]:
df3 = df.iloc[indices_division[2]:indices_division[3], :]
df3 = df3.dropna(axis=0, how='all')
df3 = df3.dropna(axis=1, how='all')
df3

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
53,Type,NaN,KSE,Buy,NaN,NaN
54,ISIN Code / Abbr. Code,NaN,KR7007340003 / A007340,NaN,NaN,NaN
55,Security Description,NaN,DN AUTOMOTIVE,NaN,NaN,NaN
57,No. of Shares / Price,NaN,NaN,27,/ KRW,91900.0
58,NaN,NaN,NaN,37,/ KRW,92000.0
59,NaN,NaN,NaN,36,/ KRW,92100.0
60,NaN,NaN,NaN,17,/ KRW,92200.0
61,NaN,NaN,NaN,155,/ KRW,92300.0
62,NaN,NaN,NaN,6,/ KRW,92400.0
63,NaN,NaN,NaN,70,/ KRW,92500.0


In [28]:
df4 = df.iloc[indices_division[3]:indices_division[4], :]
df4 = df4.dropna(axis=0, how='all')
df4 = df4.dropna(axis=1, how='all')
df4

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
87,Type,NaN,KSE,Buy,NaN,NaN
88,ISIN Code / Abbr. Code,NaN,KR7271560005 / A271560,NaN,NaN,NaN
89,Security Description,NaN,ORION,NaN,NaN,NaN
91,No. of Shares / Price,NaN,NaN,43,/ KRW,87400.0
92,NaN,NaN,NaN,90,/ KRW,87500.0
93,NaN,NaN,NaN,81,/ KRW,87600.0
94,NaN,NaN,NaN,23,/ KRW,87700.0
95,NaN,NaN,NaN,23,/ KRW,87800.0
96,NaN,NaN,NaN,75,/ KRW,87900.0
97,NaN,NaN,NaN,25,/ KRW,88000.0


In [11]:
list(df.iloc[:, 0])

[nan,
 '9th Fl. Samsung Electronics Bldg., ',
 '11, 74-gil Seocho-daero Seocho-gu, Seoul 06620, ',
 'Republic of Korea',
 nan,
 ' Trade Confirmation',
 nan,
 'Date',
 nan,
 'From',
 'To',
 nan,
 'Trade Date :',
 'Settlement Date :',
 nan,
 nan,
 'Our Reference No. :',
 'Korea IRC No. :',
 'A/C Name :',
 nan,
 'Type',
 'ISIN Code / Abbr. Code',
 'Security Description',
 nan,
 'No. of Shares / Price',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Total No. of Shares',
 'Average Price',
 'Considerations',
 'Commission',
 'Sales Tax',
 'Capital Gains Tax',
 'Net Amount',
 nan,
 'Type',
 'ISIN Code / Abbr. Code',
 'Security Description',
 nan,
 'No. of Shares / Price',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Total No. of Shares',
 'Average Price',
 'Considerations',
 'Commission',
 'Sales Tax',
 'Capital Gains Tax',
 'Net Amount'

In [13]:
list(df.iloc[:, 2])

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Shs',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Shs',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Shs',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 'KRW',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]